In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/SkinFormer"

/content/drive/MyDrive/SkinFormer


In [ ]:
%cd "/content/drive/MyDrive/UNSW/SkinFormer"


/content/drive/.shortcut-targets-by-id/1jbMn8Vv-NOdyZv5qQRqYRYo-QlNIhF9S/SkinFormer


In [3]:
!pip install -q einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.7 MB/s eta 0:00:00


In [ ]:
!python Train_DeiT-B_LRP.py --config configs.yml

In [5]:
!python DisTranPrune.py --config configs.yml

CUDA is available: True 

Starting... 

{'train': 12809, 'val': 3203}
{'train': 12809, 'val': 3203}
Weights load from /content/drive/MyDrive/SkinFormer/Outputs/LRP/DeiT-S_LRP_checkpoint_BASE.pth
Weights load from /content/drive/MyDrive/SkinFormer/Outputs/LRP/DeiT-S_LRP_checkpoint_6SA_detection.pth
+++++++++++++++++++++++++++++ Pruning Iteration 0 +++++++++++++++++++++++++++++
Generating masks: 100% 500/500 [14:20<00:00,  1.72s/it]

Final Validation results for DeiT-S_LRP_PIter0: Accuracy: 0.8607555416796753  Balanced Accuracy: 0.7398370227373777 

New leading model val metrics, saving the weights...

{'acc_avg': 0.8607555416796753, 'acc_per_type': array([0.83110368, 0.84182015, 0.85610465, 0.90192644, 0.91585761,
       0.84210526]), 'PQD': 0.9074594939551154, 'DPM': 0.5370286993441433, 'EOM': 0.6680385670499175, 'AUC': -1}
Checkpoint saved: /content/drive/MyDrive/SkinFormer/Outputs/LRP/DeiT-S_LRP_checkpoint_prune_Iter=0.pth
This iteration took 29m 53s
+++++++++++++++++++++++++++++ Pru

# Playground

In [1]:
import torch

In [2]:
SA_attr = torch.rand((5, 5))
SA_attr

tensor([[0.2135, 0.1482, 0.8024, 0.6604, 0.2994],
        [0.5955, 0.6628, 0.2646, 0.1552, 0.3052],
        [0.2261, 0.8401, 0.5397, 0.4306, 0.2331],
        [0.4727, 0.3276, 0.1478, 0.4761, 0.3077],
        [0.8109, 0.7738, 0.5994, 0.2800, 0.3688]])

In [4]:
main_attr = torch.rand((5, 5))
main_attr

tensor([[0.6277, 0.1743, 0.0142, 0.8937, 0.3596],
        [0.2825, 0.4220, 0.6265, 0.1029, 0.5229],
        [0.9167, 0.2177, 0.0236, 0.7422, 0.1904],
        [0.4376, 0.7953, 0.4486, 0.4329, 0.1755],
        [0.0963, 0.0795, 0.9038, 0.8088, 0.9243]])

In [5]:
retain_rate = 0.8


flat_tensor = main_attr.flatten()

threshold = torch.quantile(flat_tensor, 1-retain_rate)

mask1 = (main_attr < threshold).float()
mask1

tensor([[0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.]])

In [8]:
can_be_pruned = SA_attr * mask1
can_be_pruned
can_be_pruned_flt = can_be_pruned.flatten()

k = int(0.6*mask1.sum())
top_k_values, top_k_indices = torch.topk(can_be_pruned_flt, k)
mask2 = torch.ones_like(can_be_pruned_flt)
mask2[top_k_indices] = 0
mask2 = mask2.reshape((5, 5))
mask2

tensor([[1., 1., 0., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1.]])

In [10]:
torch.ones((2,3,4,5)).sum()

tensor(120.)